In [1]:
pip install tensorflow numpy pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention
import numpy as np
import pandas as pd

In [3]:
#load data
def load_data(source_path, target_path):
    with open(source_path, 'r', encoding='utf-8') as f:
        sources = f.readlines()
    with open(target_path, 'r', encoding='utf-8') as f:
        targets = f.readlines()
    return sources, targets

train_sources, train_targets = load_data(
    "C:\\Users\\91807\\OneDrive\\Desktop\\TOC\\writingPrompts\\train.wp_source",
    "C:\\Users\\91807\\OneDrive\\Desktop\\TOC\\writingPrompts\\train.wp_target"
)
valid_sources, valid_targets = load_data(
    "C:\\Users\\91807\\OneDrive\\Desktop\\TOC\\writingPrompts\\valid.wp_source",
    "C:\\Users\\91807\\OneDrive\\Desktop\\TOC\\writingPrompts\\valid.wp_target"
)
test_sources, test_targets = load_data(
    "C:\\Users\\91807\\OneDrive\\Desktop\\TOC\\writingPrompts\\test.wp_source",
    "C:\\Users\\91807\\OneDrive\\Desktop\\TOC\\writingPrompts\\test.wp_target"
)

In [4]:
#preprocess data
def preprocess_data(sources, targets, max_vocab_size=50000, max_seq_length=100):
    tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
    tokenizer.fit_on_texts(sources + targets)

    source_sequences = tokenizer.texts_to_sequences(sources)
    target_sequences = tokenizer.texts_to_sequences(targets)

    source_padded = pad_sequences(source_sequences, maxlen=max_seq_length, padding='post')
    target_padded = pad_sequences(target_sequences, maxlen=max_seq_length+1, padding='post')

    return source_padded, target_padded, tokenizer

train_source_padded, train_target_padded, tokenizer = preprocess_data(train_sources, train_targets)
valid_source_padded, valid_target_padded, _ = preprocess_data(valid_sources, valid_targets)
test_source_padded, test_target_padded, _ = preprocess_data(test_sources, test_targets)

In [5]:
#BUILD MODEL
def build_model(max_vocab_size, embedding_dim, hidden_units, max_seq_length):
    # Encoder
    encoder_inputs = Input(shape=(max_seq_length,))
    encoder_embedding = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(max_seq_length,))
    decoder_embedding = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

    # Attention
    attention = Attention()([decoder_outputs, encoder_outputs])
    decoder_concat = tf.concat([decoder_outputs, attention], axis=-1)

    # Dense Layer
    decoder_dense = Dense(max_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_concat)

    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model(max_vocab_size=50000, embedding_dim=256, hidden_units=512, max_seq_length=100)
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 256)             1280000   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 embedding_1 (Embedding)     (None, 100, 256)             1280000   ['input_2[0][0]']       

In [ ]:
history = model.fit(
    [train_source_padded, train_target_padded[:, :-1]],  # Input shape: (None, 100)
    train_target_padded[:, 1:],                          # Output shape: (None, 100)
    batch_size=64,
    epochs=10,
    validation_data=([valid_source_padded, valid_target_padded[:, :-1]], valid_target_padded[:, 1:])
)

Epoch 1/10


  58/4260 [..............................] - ETA: 100:29:10 - loss: 7.3439 - accuracy: 0.0625

In [ ]:
model.save("story_generation_model.h5")

In [ ]:
#load and generate story
from tensorflow.keras.models import load_model
model = load_model("story_generation_model.h5")

def generate_story(prompt, model, tokenizer, max_seq_length=100):
    prompt_seq = tokenizer.texts_to_sequences([prompt])
    prompt_padded = pad_sequences(prompt_seq, maxlen=max_seq_length, padding='post')

    # Initialize decoder input with start token
    decoder_input = np.zeros((1, max_seq_length))
    decoder_input[0, 0] = tokenizer.word_index['<start>']

    # Generate story
    for i in range(1, max_seq_length):
        predictions = model.predict([prompt_padded, decoder_input])
        predicted_id = np.argmax(predictions[0, i-1, :])
        decoder_input[0, i] = predicted_id
        if predicted_id == tokenizer.word_index['<end>']:
            break

    # Convert sequence to text
    story = tokenizer.sequences_to_texts(decoder_input)[0]
    return story

In [ ]:
#flask api
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data.get('prompt')
    story = generate_story(prompt, model, tokenizer)
    return jsonify({"story": story})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)